#  3. 특정 업체(화양156)의 메뉴 긍정 리뷰 분석

## 목차 

 
    1. 화양156 데이터 전처리
    2. 화양156의 긍정 리뷰 데이터 추출
    3. 긍정 리뷰 데이터의 LSA이용한 유사도 측정 
    4. 코사인 유사도 이용하여 특정 메뉴에 대한 유사도 높은 리뷰 추출
    5. 결과

In [1]:
import pandas as pd
import networkx as nx 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

import re
from wordcloud import WordCloud 
from wordcloud import STOPWORDS

from konlpy.tag import *
from konlpy.tag import Okt
import konlpy
import nltk
import pandas as pd
from math import log

import numpy as np
from numpy import array
from numpy import dot
from numpy.linalg import norm
from scipy.linalg import svd

In [2]:
# 윈도우의 한글 폰트 설정
plt.rc('font', family = 'Malgun Gothic')

# 시각화 그래프가 노트북 안에 보이게 하기
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False

# warning 메세지 
warnings.simplefilter("ignore")

## 1. 화양156 데이터 전처리

In [3]:
# data확인
df = pd.read_csv('./data/data_review.csv',encoding='utf-8')
df

,업체명,카테고리,메뉴,맛,양,배달,리뷰,date
0,전주석쇠불고기-본점,한식,파절이매콤통삼겹（2~3인）（공기밥2＋김치찌개＋밑반찬＋쌈）/1,5.0,5.0,5.0,자주시켜먹는 단골집인데 항상변치않고 맛있습니다! !!,2017년 12월 6일 수요일
1,전주석쇠불고기-본점,한식,통삼겹살 2人（고기＋공기밥2＋김치찌개＋쌈＋밑반찬）/1(추가 선택(고기 추가)),5.0,5.0,5.0,"배달 시간도 오래걸리지 않고, 양이 적을거 같아서 고기 추가를 했는데..안해도 됐었...",2017년 9월 30일 토요일
2,전주석쇠불고기-본점,한식,통삼겹살 2人（고기＋공기밥2＋김치찌개＋쌈＋밑반찬）/1,NaN,NaN,NaN,굿굿,2017년 9월 23일 토요일
3,전주석쇠불고기-본점,한식,통삼겹（小／500g）（냉열무국수 or 냉열무우동 or 비빔열무국수＋쌈）/1(메뉴 선...,5.0,5.0,5.0,배달도빠르고맛나요ㅁ,2018년 3월 13일 화요일
4,전주석쇠불고기-본점,한식,통삼겹（3~4인）（김치찌개＋공기밥3＋밑반찬＋쌈）/1,5.0,5.0,5.0,찌개와 통삼겹 맛있는 5찬. 많이 먹는편이라 3인분짜리 주문했는데... 배터지는줄 ...,2018년 2월 27일 화요일
...,...,...,...,...,...,...,...,...
28912,동강,중식,NaN,NaN,NaN,NaN,좋아요,2013년 12월 31일 화요일
28913,동강,중식,NaN,NaN,NaN,NaN,맛있ㅇㅓ욧,2013년 12월 29일 일요일
28914,동강,중식,NaN,NaN,NaN,NaN,맛이게다,2013년 12월 25일 수요일
28915,동강,중식,NaN,NaN,NaN,NaN,여기 맛있어요 이근처 탕수육은 냄새?ㅜㅜ돼지냄새?나는 곳이 너무 많은데 여긴안그래요...,2013년 12월 10일 화요일


## 2. 화양156의 긍정 리뷰 데이터 추출

In [4]:
# 화양156 리뷰 데이터
df_HW = df[df['업체명'] == '화양156']

# 화양156 점수있는 데이터
df_HW = df_HW[df_HW['맛'].notnull() & df_HW['양'].notnull() & df_HW['배달'].notnull()]

In [5]:
df_HW['총점'] = df_HW['맛']+ df_HW['양']+df_HW['배달']
df_HW_Good = df_HW[ ( df_HW['총점'] >= 12 )& (df_HW['맛'] >= 4 )]
df_HW_Good['리뷰'] =df_HW_Good['리뷰'].str.replace('[^a-zA-Z0-9가-힣\s]', '', regex=True)
df_HW_Good

,업체명,카테고리,메뉴,맛,양,배달,리뷰,date,총점
23216,화양156,분식,"화양세트3（떡볶이＋튀김＋주먹밥＋음료）/1(토핑 추가 선택(차돌),맛 선택(매운맛),...",5.0,5.0,5.0,정말 푸짐하고 맛있는데다 샐러드서비스에 친절하시기까지 앞으로 떡볶이는 여기서만 시킬...,2018년 4월 3일 화요일,15.0
23218,화양156,분식,"화양세트3（떡볶이＋튀김＋주먹밥＋음료）/1(토핑 추가 선택(떡 추가),맛 선택(매운맛...",5.0,5.0,5.0,맛있어요 생각보다 양이 많아서 놀랐습니다 다음에도 또 시킬게오,2018년 5월 5일 토요일,15.0
23220,화양156,분식,"화양세트3（떡볶이＋튀김＋주먹밥＋음료）/1(맛 선택(순한맛),떡볶이 선택(화양떡볶이))",5.0,5.0,5.0,맛도 있고 양도 많고 배달해주시는분도 친절하시고 배달도 빠르고 모든게 좋았습니다,2018년 5월 22일 화요일,15.0
23222,화양156,분식,"화양세트3（떡볶이＋튀김＋주먹밥＋음료）/1(맛 선택(순한맛),떡볶이 선택(화양떡볶이))",5.0,5.0,5.0,맛있어요 담에도 또 시켜먹을께요,2018년 4월 20일 금요일,15.0
23223,화양156,분식,"화양세트3（떡볶이＋튀김＋주먹밥＋음료）/1(맛 선택(순한맛),떡볶이 선택(화양떡볶이))",5.0,4.0,5.0,배달도 빠르고 너무너무 맛 있습니다 언제부턴가 떡뽁이는 이집아니면 못먹겠어요,2018년 4월 19일 목요일,14.0
...,...,...,...,...,...,...,...,...,...
23759,화양156,분식,"YOLO 떡볶이/1(맛 선택(순한맛),떡볶이 선택(화양떡볶이)),김말이 /1,야채튀...",5.0,5.0,5.0,튀김이랑 같이는 처음 시켜먹어보고있는데튀김이 정말 맛있네요먹다가 리뷰씁니당,2018년 5월 24일 목요일,15.0
23761,화양156,분식,NaN,5.0,5.0,5.0,처음시켜먹어봤는데 진짜 앞으로 자주시켜 먹을거같아요 맛양 둘다 만족했어요,2018년 3월 31일 토요일,15.0
23765,화양156,분식,NaN,5.0,5.0,5.0,주문누락때문에 오래걸렸지만 전화드리자마자 10분만에왔네요 일부러늦으신것도 아닌데 서...,2017년 12월 10일 일요일,15.0
23766,화양156,분식,NaN,5.0,5.0,5.0,4번째먹는데 언제나 맛잇어용,2017년 11월 12일 일요일,15.0


In [10]:
ko_text = ''
for i in df_HW_Good['리뷰']:
    ko_text += i+' '

## 3. 긍정 리뷰 데이터의 LSA이용한 유사도 측정

In [12]:
words = konlpy.tag.Twitter().pos(ko_text)

parser = nltk.RegexpParser('NP: {<N.*>*<Suffix>?}')
chunks = parser.parse(words)

noun_chunks = []
for subtree in chunks.subtrees():
    if subtree.label()=='NP':
        noun_chunks.append(' '.join((e[0] for e in list(subtree))))
for i in noun_chunks:
    texts = i+' '

In [14]:
df_HW_Good_reviews = pd.DataFrame(columns= (set(noun_chunks)))
df_HW_Good_reviews
for i in df_HW_Good['리뷰']:
    row = []
    one_review = i
    for noun in df_HW_Good_reviews.columns:
        row.append(one_review.count(noun))
    df_HW_Good_reviews.loc[len(df_HW_Good_reviews)] = row
df_HW_Good_reviews =  df_HW_Good_reviews.astype('float')

In [22]:
df_HW_Terms_reivews = df_HW_Good_reviews.T
from math import log
N = 406
# df_t = []
idf = []
for tf in df_HW_Terms_reivews.values:
    tf = tf.tolist().count(0)
    idf.append(round(log( N / (1+(406-tf)) ),4))
idf[:5]

[5.3132, 4.6201, 6.0064, 5.3132, 5.3132]

In [23]:
import numpy as np
idf= np.array(idf)
df_idf = pd.DataFrame(data=idf.transpose(), index=df_HW_Terms_reivews.index, columns=['idf'])

In [24]:
df_idf

,idf
맛짐 단 포장,5.3132
배달 떡볶이,4.6201
요기 잘먹엇습니,6.0064
돈까스 샐러드 김치볶음밥,5.3132
걸 계속,5.3132
...,...
소스 양,5.3132
순간 오지,5.3132
덜,3.9269
오픈 한지 얼마,6.0064


In [25]:
# TF-IDF matrix 생성
tf_idf = pd.DataFrame(index = df_HW_Terms_reivews.index, columns=range(N))
tf_idf
for i in range(len(df_HW_Terms_reivews.index)):
    tf_idf.iloc[i] = round(df_HW_Terms_reivews.iloc[i]*idf[i],4)

In [26]:
tf_idf

,0,1,2,3,4,5,6,7,8,9,...,396,397,398,399,400,401,402,403,404,405
맛짐 단 포장,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
배달 떡볶이,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
요기 잘먹엇습니,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
돈까스 샐러드 김치볶음밥,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
걸 계속,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
소스 양,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
순간 오지,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
덜,0,0,0,0,0,0,0,0,0,3.9269,...,0,0,0,0,0,0,0,0,0,0
오픈 한지 얼마,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# SVD 계산하기 위해 df를 numpy로, data type을 float으로 설정했습니다.
TF_IDF = tf_idf.to_numpy( dtype='float')
TF_IDF

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
# Singular-Value Decomposition

# SVD
U, sigma, VT = svd(TF_IDF)

print('U : ' ,U,'\n')
print('시그마 : ',sigma,'\n')
print('V^T : ',VT,'\n')

U :  [[-9.14122400e-03 -2.22788016e-02  8.14861030e-02 ... -4.31806346e-02
   0.00000000e+00  3.81819154e-02]
 [-1.48594282e-02  2.18383639e-02 -1.13953573e-02 ...  5.44982561e-02
   0.00000000e+00  4.30233716e-03]
 [ 2.86229374e-17  1.73472348e-17 -8.67361738e-18 ... -4.83086455e-03
   0.00000000e+00 -3.39788680e-02]
 ...
 [-4.31703446e-02 -2.40606171e-02  1.23229030e-01 ...  4.91506138e-01
   0.00000000e+00 -1.39740196e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [-1.23969217e-02  8.57482062e-03  2.56185091e-02 ... -7.07804960e-03
   0.00000000e+00  3.20514733e-01]] 

시그마 :  [8.23602331e+01 4.42009094e+01 3.92972901e+01 3.70861574e+01
 3.53139612e+01 3.46395858e+01 3.37578572e+01 3.30916809e+01
 3.26539452e+01 3.20099472e+01 3.10819887e+01 3.06656508e+01
 2.96140174e+01 2.92151662e+01 2.88743016e+01 2.84880949e+01
 2.81907669e+01 2.77916155e+01 2.72951990e+01 2.67224578e+01
 2.63100665e+01 2.60107459e+01 2.57922462e+01

In [29]:
Sigma = np.diag(sigma)
Sigma

U_df = pd.DataFrame(U)
U_df

Sigma_df = pd.DataFrame(Sigma)
Sigma_df

,0,1,2,3,4,5,6,7,8,9,...,396,397,398,399,400,401,402,403,404,405
0,82.360233,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.000000,44.200909,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.000000,0.000000,39.29729,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.000000,0.000000,0.00000,37.086157,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.000000,0.000000,0.00000,0.000000,35.313961,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.799790e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
402,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,6.799790e-15,0.000000e+00,0.000000e+00,0.000000e+00
403,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,6.799790e-15,0.000000e+00,0.000000e+00
404,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,6.799790e-15,0.000000e+00


In [30]:
# pick two left singular vectors
U_df.iloc[:, :2]

,0,1
0,-9.141224e-03,-2.227880e-02
1,-1.485943e-02,2.183836e-02
2,2.862294e-17,1.734723e-17
3,-1.541124e-02,-1.044443e-01
4,-6.361860e-03,1.782723e-03
...,...,...
1180,-5.955303e-03,3.322774e-03
1181,-2.188635e-03,3.122645e-03
1182,-4.317034e-02,-2.406062e-02
1183,0.000000e+00,0.000000e+00


In [31]:
# The first two singular values
Sigma_df.iloc[:2,:2]

,0,1
0,82.360233,0.000000
1,0.000000,44.200909


In [32]:
LSM= U_df.iloc[:, :2].dot(Sigma_df.iloc[:2,:2])
LSM

,0,1
0,-7.528733e-01,-9.847433e-01
1,-1.223826e+00,9.652755e-01
2,2.357392e-15,7.667636e-16
3,-1.269274e+00,-4.616532e+00
4,-5.239642e-01,7.879799e-02
...,...,...
1180,-4.904802e-01,1.468696e-01
1181,-1.802565e-01,1.380237e-01
1182,-3.555520e+00,-1.063501e+00
1183,0.000000e+00,0.000000e+00


In [33]:
LSM

,0,1
0,-7.528733e-01,-9.847433e-01
1,-1.223826e+00,9.652755e-01
2,2.357392e-15,7.667636e-16
3,-1.269274e+00,-4.616532e+00
4,-5.239642e-01,7.879799e-02
...,...,...
1180,-4.904802e-01,1.468696e-01
1181,-1.802565e-01,1.380237e-01
1182,-3.555520e+00,-1.063501e+00
1183,0.000000e+00,0.000000e+00


In [34]:
LSM = LSM.set_axis(df_HW_Terms_reivews.index, axis='index')

In [35]:
LSM

,0,1
맛짐 단 포장,-7.528733e-01,-9.847433e-01
배달 떡볶이,-1.223826e+00,9.652755e-01
요기 잘먹엇습니,2.357392e-15,7.667636e-16
돈까스 샐러드 김치볶음밥,-1.269274e+00,-4.616532e+00
걸 계속,-5.239642e-01,7.879799e-02
...,...,...
소스 양,-4.904802e-01,1.468696e-01
순간 오지,-1.802565e-01,1.380237e-01
덜,-3.555520e+00,-1.063501e+00
오픈 한지 얼마,0.000000e+00,0.000000e+00


In [36]:
#코사인유사도를 사용해 모든 similarity 계산  
for n in range(len(LSM)-1):
    for n2 in range(n+1, len(LSM)):
        a,b = LSM.iloc[n], LSM.iloc[n2]
        CS = round(dot(a, b)/(norm(a)*norm(b)),4)
# 0.9 이상의 코사인유사도를 가지는 단어 출력
#         if CS >= 0.9 :
#             print('{}-{} 코사인 유사도 : '.format(LSM.index[n],LSM.index[n2]),CS,'\n')
                

In [37]:
df_HW_Terms_reivews_CS = pd.DataFrame(columns=df_HW_Terms_reivews.index, index=df_HW_Terms_reivews.index)
for n in range(len(LSM)-1):
    for n2 in range(n+1, len(LSM)):
        a,b = LSM.iloc[n], LSM.iloc[n2]
        df_HW_Terms_reivews_CS.iloc[n,n2] = round(dot(a, b)/(norm(a)*norm(b)),4)

In [38]:
df_HW_Terms_reivews_CS

,맛짐 단 포장,배달 떡볶이,요기 잘먹엇습니,돈까스 샐러드 김치볶음밥,걸 계속,의사,먹기 띡,통 새우 튀김,확인,나머진,...,가라아게,담,요청 사항 정말,주황,도착,소스 양,순간 오지,덜,오픈 한지 얼마,용기
맛짐 단 포장,NaN,-0.0151,-0.8233,0.927,0.4825,-0.1781,-0.0544,-0.6689,0.2748,0.5476,...,0.4559,0.2125,NaN,0.5476,-0.0284,0.354,-0.0007,0.8095,NaN,0.2929
배달 떡볶이,NaN,NaN,-0.5551,-0.389,0.8685,0.9866,0.9992,-0.7332,0.9572,0.8284,...,0.883,0.9738,NaN,0.8284,0.9999,0.9298,0.9999,0.5748,NaN,0.9516
요기 잘먹엇습니,NaN,NaN,NaN,-0.5503,-0.8944,-0.4119,-0.522,0.9726,-0.772,-0.9258,...,-0.8805,-0.7296,NaN,-0.9258,-0.544,-0.8223,-0.567,-0.9997,NaN,-0.7839
돈까스 샐러드 김치볶음밥,NaN,NaN,NaN,NaN,0.1188,-0.5342,-0.4249,-0.3412,-0.1058,0.1939,...,0.0889,-0.1695,NaN,0.1939,-0.4012,-0.0226,-0.3757,0.5303,NaN,-0.087
걸 계속,NaN,NaN,NaN,NaN,NaN,0.776,0.8484,-0.9738,0.9748,0.9971,...,0.9995,0.9584,NaN,0.9971,0.8619,0.99,0.8756,0.9048,NaN,0.9788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
소스 양,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.935,0.8356,NaN,0.9979
순간 오지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5865,NaN,0.9559
덜,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7984
오픈 한지 얼마,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_HW_Terms_reivews_CS = df_HW_Terms_reivews_CS.fillna(df_HW_Terms_reivews_CS.T)

In [41]:
df_HW_Terms_reivews_CS_a = df_HW_Terms_reivews_CS.copy()


## 4. 코사인 유사도 이용하여 특정 메뉴에 대한 유사도 높은 리뷰 추출

In [42]:
df_HW_Terms_reivews_CS_a['떡볶이 샐러드 튀김'].sum()

717.1443999999996

In [43]:
 df_HW_Terms_reivews_CS_a.columns

Index(['맛짐 단 포장', '배달 떡볶이', '요기 잘먹엇습니', '돈까스 샐러드 김치볶음밥', '걸 계속', '의사', '먹기 띡',
       '통 새우 튀김', '확인', '나머진',
       ...
       '가라아게', '담', '요청 사항 정말', '주황', '도착', '소스 양', '순간 오지', '덜', '오픈 한지 얼마',
       '용기'],
      dtype='object', length=1185)

In [44]:
for i in df_HW_Terms_reivews_CS_a.columns:
    if df_HW_Terms_reivews_CS_a[i].sum()==0:
        df_HW_Terms_reivews_CS_a= df_HW_Terms_reivews_CS_a.drop(columns=i, index=i)
df_HW_Terms_reivews_CS_a

,맛짐 단 포장,배달 떡볶이,요기 잘먹엇습니,돈까스 샐러드 김치볶음밥,걸 계속,의사,먹기 띡,통 새우 튀김,확인,나머진,...,리뷰 전,살짝 더,가라아게,담,주황,도착,소스 양,순간 오지,덜,용기
맛짐 단 포장,NaN,-0.0151,-0.8233,0.927,0.4825,-0.1781,-0.0544,-0.6689,0.2748,0.5476,...,0.6786,0.2457,0.4559,0.2125,0.5476,-0.0284,0.354,-0.0007,0.8095,0.2929
배달 떡볶이,-0.0151,NaN,-0.5551,-0.389,0.8685,0.9866,0.9992,-0.7332,0.9572,0.8284,...,0.7242,0.9655,0.883,0.9738,0.8284,0.9999,0.9298,0.9999,0.5748,0.9516
요기 잘먹엇습니,-0.8233,-0.5551,NaN,-0.5503,-0.8944,-0.4119,-0.522,0.9726,-0.772,-0.9258,...,-0.9756,-0.7525,-0.8805,-0.7296,-0.9258,-0.544,-0.8223,-0.567,-0.9997,-0.7839
돈까스 샐러드 김치볶음밥,0.927,-0.389,-0.5503,NaN,0.1188,-0.5342,-0.4249,-0.3412,-0.1058,0.1939,...,0.3536,-0.1358,0.0889,-0.1695,0.1939,-0.4012,-0.0226,-0.3757,0.5303,-0.087
걸 계속,0.4825,0.8685,-0.8944,0.1188,NaN,0.776,0.8484,-0.9738,0.9748,0.9971,...,0.9708,0.9676,0.9995,0.9584,0.9971,0.8619,0.99,0.8756,0.9048,0.9788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
도착,-0.0284,0.9999,-0.544,-0.4012,0.8619,0.9887,0.9997,-0.7241,0.9533,0.8209,...,0.715,0.962,0.8767,0.9707,0.8209,NaN,0.9248,0.9996,0.5639,0.9474
소스 양,0.354,0.9298,-0.8223,-0.0226,0.99,0.8573,0.9146,-0.932,0.9965,0.9764,...,0.9271,0.9936,0.9938,0.9891,0.9764,0.9248,NaN,0.935,0.8356,0.9979
순간 오지,-0.0007,0.9999,-0.567,-0.3757,0.8756,0.9841,0.9986,-0.7429,0.9613,0.8363,...,0.734,0.9692,0.8897,0.977,0.8363,0.9996,0.935,NaN,0.5865,0.9559
덜,0.8095,0.5748,-0.9997,0.5303,0.9048,0.4335,0.5422,-0.9779,0.7869,0.9345,...,0.9806,0.768,0.8916,0.7457,0.9345,0.5639,0.8356,0.5865,NaN,0.7984


In [45]:
term_list =[]
for i in df_HW_Terms_reivews_CS_a.columns:
    if '차돌' in i:
        term_list.append(i)
term_list 

['차돌박이 떡뽁이', '차돌', '차돌 간장 모두', '순서대로 차돌']

In [46]:
df_HW_Terms_reivews_CS_a['차돌'].max()

1.0

In [47]:
for i in term_list :
    print('"'+i+'"','키워드와 유사한 키워드')
    print(list(df_HW_Terms_reivews_CS_a[ df_HW_Terms_reivews_CS_a[i] ==df_HW_Terms_reivews_CS_a[i].max()].index),'\n')

"차돌박이 떡뽁이" 키워드와 유사한 키워드
['따끈따끈 떡볶이', '완전 중됵', '은근', '먹기 전', '양인 것', '단무지', '밸런스', '주심', '먾다', '단 크림소스', '꼭 다시', '직접 배달', '떡볶이 조합'] 

"차돌" 키워드와 유사한 키워드
['보통 맛', '까르보나라 처음', '처음 주문', '떡볶이', '이유', '순삭 돈가스', '습', '언제', '대도', '봉투', '꼭 번창', '바로 전화', '떡볶이 국물', '것', '만족', '방금 주문', '참치', '샐러드 채소 정말', '용기'] 

"차돌 간장 모두" 키워드와 유사한 키워드
['이건', '안내', '리뷰', '역시 맛'] 

"순서대로 차돌" 키워드와 유사한 키워드
['리뷰 얼핏 보고', '계속', '떡볶이 오뎅', '옄 샐러드', '유행', '자꾸', '땐 무조건 여기', '이제 이 다음', '요양', '예전', '메뉴판', '더 배', '가라아게 서비스', '리뷰 전'] 



## 5. 결과

화양156의 데이터에서 총점(맛+양+배달) 12점 이상, 맛 4점 이상의 데이터를 긍정 리뷰 데이터로 판단했습니다.

해당 리뷰의 데이터를 전처리 후 tf-idf 생성하고 LSA를 시행했습니다.

이를 통해 각 리뷰 내용(명사 구)에 대한 코사인 유사도를 구할 수 있었고,
입력 메뉴에 대한 리뷰를 확인할 수 있었습니다.